In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, row_number
from pyspark.sql.window import Window  # <-- This import was missing
# Initialize Spark session
spark = SparkSession.builder.appName("F1 Data Analysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 22:55:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = "data/"

# Load CSV files into DataFrames with the updated path
constructor_performance_df = spark.read.csv(data_path + "Constructor_Performance.csv", header=True, inferSchema=True)
constructor_rankings_df = spark.read.csv(data_path + "Constructor_Rankings.csv", header=True, inferSchema=True)
driver_details_df = spark.read.csv(data_path + "Driver_Details.csv", header=True, inferSchema=True)
driver_rankings_df = spark.read.csv(data_path + "Driver_Rankings.csv", header=True, inferSchema=True)
lap_timings_df = spark.read.csv(data_path + "Lap_Timings.csv", header=True, inferSchema=True)
pit_stop_records_df = spark.read.csv(data_path + "Pit_Stop_Records.csv", header=True, inferSchema=True)
qualifying_results_df = spark.read.csv(data_path + "Qualifying_Results.csv", header=True, inferSchema=True)
race_results_df = spark.read.csv(data_path + "Race_Results.csv", header=True, inferSchema=True)
race_schedule_df = spark.read.csv(data_path + "Race_Schedule.csv", header=True, inferSchema=True)
race_status_df = spark.read.csv(data_path + "Race_Status.csv", header=True, inferSchema=True)
season_summaries_df = spark.read.csv(data_path + "Season_Summaries.csv", header=True, inferSchema=True)
sprint_race_results_df = spark.read.csv(data_path + "Sprint_Race_Results.csv", header=True, inferSchema=True)
team_details_df = spark.read.csv(data_path + "Team_Details.csv", header=True, inferSchema=True)
track_information_df = spark.read.csv(data_path + "Track_Information.csv", header=True, inferSchema=True)


In [3]:
# Join race_results_df with driver_details_df on driverId

from pyspark.sql.functions import concat_ws

# Concatenate forename and surname to create a full name
driver_details_with_full_name = driver_details_df.withColumn("driverName", concat_ws(" ", driver_details_df["forename"], driver_details_df["surname"]))

# Top 5 Drivers with the Most Wins and their names
top_drivers_by_wins = race_results_df.filter(race_results_df["position"] == 1) \
    .groupBy("driverId") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(5)

# Join with driver details to get the driver names
top_drivers_with_names = top_drivers_by_wins.join(driver_details_with_full_name, "driverId") \
    .select("driverName", "count") \
    .orderBy("count", ascending=False)

top_drivers_with_names.show()

+------------------+-----+
|        driverName|count|
+------------------+-----+
|    Lewis Hamilton|  104|
|Michael Schumacher|   91|
|    Max Verstappen|   61|
|  Sebastian Vettel|   53|
|       Alain Prost|   51|
+------------------+-----+



In [4]:
# Concatenate forename and surname to create a full name
driver_details_with_full_name = driver_details_df.withColumn("driverName", concat_ws(" ", driver_details_df["forename"], driver_details_df["surname"]))

# Top 5 Drivers with the Most Wins
top_drivers_by_wins = race_results_df.filter(race_results_df["position"] == 1) \
    .groupBy("driverId") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(5)

# Join with driver details to get the driver names
top_drivers_with_names = top_drivers_by_wins.join(driver_details_with_full_name, "driverId") \
    .select("driverId", "driverName", "count") \
    .orderBy("count", ascending=False)

# Load the race schedule for race dates and names
# Assuming race_schedule_df has "Race_ID", "Race_Name", "Date", and "Year" columns
# Load the race schedule for race dates and names
# Updated to use the correct column names: "raceId", "name", "date", "year"
# Load the race schedule for race dates and names
# Updated to use the correct column names: "raceId", "name", "date", "year"
race_schedule_with_info = race_schedule_df.select("raceId", "name", "date", "year")

# Filter for top 5 drivers
top_5_drivers = top_drivers_with_names.select("driverId").rdd.flatMap(lambda x: x).collect()

# Rename raceId in one of the DataFrames to avoid ambiguity
race_results_df_renamed = race_results_df.withColumnRenamed("raceId", "raceResultId")

# Get all the race wins for the top 5 drivers
race_wins = race_results_df_renamed.filter(race_results_df_renamed["position"] == 1) \
    .filter(race_results_df_renamed["driverId"].isin(top_5_drivers)) \
    .join(race_schedule_with_info, race_results_df_renamed["raceResultId"] == race_schedule_with_info["raceId"]) \
    .select("driverId", "raceId", "name", "year", "date", "position")

# Add row numbers to each win per driver (ordered by date descending) to get the last 5 wins
window_spec = Window.partitionBy("driverId").orderBy(race_wins["date"].desc())
race_wins_with_row_number = race_wins.withColumn("row_number", row_number().over(window_spec))

# Filter to only keep the last 5 wins for each driver
last_5_wins = race_wins_with_row_number.filter(race_wins_with_row_number["row_number"] <= 5)

# Join with driver details to get driver names
last_5_wins_with_names = last_5_wins.join(driver_details_with_full_name, "driverId") \
    .select("driverName", "raceId", "name", "year", "date", "row_number") \
    .orderBy("driverName", "date", ascending=False)

# Show the results
last_5_wins_with_names.show(truncate=False)

+------------------+------+-------------------------+----+----------+----------+
|driverName        |raceId|name                     |year|date      |row_number|
+------------------+------+-------------------------+----+----------+----------+
|Sebastian Vettel  |1024  |Singapore Grand Prix     |2019|2019-09-22|1         |
|Sebastian Vettel  |1001  |Belgian Grand Prix       |2018|2018-08-26|2         |
|Sebastian Vettel  |998   |British Grand Prix       |2018|2018-07-08|3         |
|Sebastian Vettel  |995   |Canadian Grand Prix      |2018|2018-06-10|4         |
|Sebastian Vettel  |990   |Bahrain Grand Prix       |2018|2018-04-08|5         |
|Michael Schumacher|68    |Chinese Grand Prix       |2006|2006-10-01|1         |
|Michael Schumacher|67    |Italian Grand Prix       |2006|2006-09-10|2         |
|Michael Schumacher|64    |German Grand Prix        |2006|2006-07-30|3         |
|Michael Schumacher|63    |French Grand Prix        |2006|2006-07-16|4         |
|Michael Schumacher|62    |U

In [5]:
# Aggregate points for each constructor
top_constructors = constructor_performance_df.groupBy("constructorId") \
    .agg({"points": "sum"}) \
    .orderBy("sum(points)", ascending=False)

# Join with team details to get constructor names
top_constructors_with_names = top_constructors.join(team_details_df, "constructorId") \
    .select("name", "sum(points)") \
    .orderBy("sum(points)", ascending=False)

top_constructors_with_names.show(truncate=False)


+--------------+-----------+
|name          |sum(points)|
+--------------+-----------+
|Ferrari       |10001.0    |
|Red Bull      |7621.0     |
|Mercedes      |7443.5     |
|McLaren       |6685.5     |
|Williams      |3630.0     |
|Renault       |1777.0     |
|Force India   |1098.0     |
|Team Lotus    |918.0      |
|Benetton      |861.5      |
|Lotus F1      |706.0      |
|Tyrrell       |621.0      |
|Brabham       |595.0      |
|Sauber        |557.0      |
|Toro Rosso    |500.0      |
|Aston Martin  |480.0      |
|Alpine F1 Team|457.0      |
|BRM           |439.0      |
|Ligier        |388.0      |
|AlphaTauri    |309.0      |
|BMW Sauber    |308.0      |
+--------------+-----------+
only showing top 20 rows



In [6]:
# Join race results with race schedule to get the year
race_times_with_year = race_results_df.join(race_schedule_df, "raceId") \
    .select("driverId", "raceId", "milliseconds", "year")

# Calculate average race time by driver and season
average_race_time = race_times_with_year.groupBy("driverId", "year") \
    .agg({"milliseconds": "avg"}) \
    .orderBy("avg(milliseconds)")

# Join with driver details to get driver names
average_race_time_with_names = average_race_time.join(driver_details_with_full_name, "driverId") \
    .select("driverName", "year", "avg(milliseconds)") \
    .orderBy("year", "avg(milliseconds)")

average_race_time_with_names.show(truncate=False)


+---------------------+----+-----------------+
|driverName           |year|avg(milliseconds)|
+---------------------+----+-----------------+
|Franco Comotti       |1950|NULL             |
|José Froilán González|1950|NULL             |
|Joie Chitwood        |1950|NULL             |
|Fred Agabashian      |1950|NULL             |
|Myron Fohr           |1950|NULL             |
|Maurice Trintignant  |1950|NULL             |
|Nello Pagani         |1950|NULL             |
|Cecil Green          |1950|NULL             |
|Brian Shawe Taylor   |1950|NULL             |
|Toni Branca          |1950|NULL             |
|Travis Webb          |1950|NULL             |
|Geoff Crossley       |1950|NULL             |
|Jerry Hoyt           |1950|NULL             |
|Raymond Sommer       |1950|NULL             |
|Sam Hanks            |1950|NULL             |
|Bill Cantrell        |1950|NULL             |
|Louis Chiron         |1950|NULL             |
|Duane Carter         |1950|NULL             |
|David Hampsh

In [8]:


# Find the fastest lap in each race and rename the aggregated column
fastest_laps = lap_timings_df.groupBy("raceId") \
    .agg({"milliseconds": "min"}) \
    .withColumnRenamed("min(milliseconds)", "milliseconds") \
    .orderBy("raceId")

# Join with the original lap timings to get the driverId based on the fastest milliseconds
fastest_laps_with_driver = fastest_laps.join(lap_timings_df, ["raceId", "milliseconds"])

# Join with driver details and race schedule to get driver names and race info
fastest_laps_with_names = fastest_laps_with_driver \
    .join(driver_details_with_full_name, "driverId") \
    .join(race_schedule_df, "raceId") \
    .select("name", "year", "raceId", "driverName", "milliseconds") \
    .orderBy("year", "milliseconds")

# Show the results
fastest_laps_with_names.show(truncate=False)



+---------------------+----+------+---------------------+------------+
|name                 |year|raceId|driverName           |milliseconds|
+---------------------+----+------+---------------------+------------+
|French Grand Prix    |1996|232   |Jacques Villeneuve   |78610       |
|Hungarian Grand Prix |1996|235   |Damon Hill           |80093       |
|European Grand Prix  |1996|227   |Damon Hill           |81363       |
|Brazilian Grand Prix |1996|225   |Damon Hill           |81547       |
|Canadian Grand Prix  |1996|231   |Jacques Villeneuve   |81916       |
|Portuguese Grand Prix|1996|238   |Jacques Villeneuve   |82873       |
|Monaco Grand Prix    |1996|229   |Jean Alesi           |85205       |
|Italian Grand Prix   |1996|237   |Michael Schumacher   |86110       |
|San Marino Grand Prix|1996|228   |Damon Hill           |88931       |
|British Grand Prix   |1996|233   |Jacques Villeneuve   |89288       |
|Argentine Grand Prix |1996|226   |Jean Alesi           |89413       |
|Austr

In [14]:
from pyspark.sql.functions import col, min, max

# Find the longest and shortest pit stops in each race and rename the aggregated columns
pit_stop_analysis = pit_stop_records_df.groupBy("raceId") \
    .agg(min("duration").alias("min_duration"), max("duration").alias("max_duration")) \
    .orderBy("raceId")

# Join with the original pit stop records to get the driverId based on the raceId and duration
pit_stop_analysis_with_driver = pit_stop_analysis.join(
    pit_stop_records_df,
    (pit_stop_analysis["raceId"] == pit_stop_records_df["raceId"]) &
    ((pit_stop_analysis["min_duration"] == pit_stop_records_df["duration"]) |
     (pit_stop_analysis["max_duration"] == pit_stop_records_df["duration"]))
).select("raceId", "driverId", "min_duration", "max_duration")

# Join with driver details and race schedule to get driver names and race info
pit_stop_analysis_with_names = pit_stop_analysis_with_driver \
    .join(driver_details_with_full_name, "driverId") \
    .join(race_schedule_df, "raceId") \
    .select("raceId", "name", "year", "driverName", "min_duration", "max_duration") \
    .orderBy("year", "min_duration")

# Show the results
pit_stop_analysis_with_names.show(truncate=False)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `min_duration` cannot be resolved. Did you mean one of the following? [`max_duration`, `raceId`].;
'Project [raceId#170, 'min_duration, max_duration#1304]
+- Project [raceId#170, max(duration)#1299 AS max_duration#1304]
   +- Aggregate [raceId#170], [raceId#170, max(duration#175) AS max(duration)#1299]
      +- Relation [raceId#170,driverId#171,stop#172,lap#173,time#174,duration#175,milliseconds#176] csv
